In [27]:
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [28]:
def order_book(month,day):
    data1 = []
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/stat_order_book/CNF14_0'+str(month)+'_'+str(day)+'_order_book_final.csv'
    data1 = pd.read_csv(datapath,sep='\t',encoding = 'utf-8')
    data_book = data1[['0','1','2','3']]
    return data_book

In [29]:
def day_time(month,day):
    data = []
    datapath = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/CN_Futures_2014.0'+str(month)+'.'+str(day)+'.csv'
    data = pd.read_csv(datapath)
    data_CNG14 = data[data.Series == 'CNG14']
    data = data_CNG14
    market_open_time = data[data['TimeStamp'].str.contains('2014-0'+str(month)+'-'+str(day)+'D09:00')].index.tolist()[0] 
    market_close_time = data[data['TimeStamp'].str.contains('2014-0'+str(month)+'-'+str(day)+'D16:00')].index.tolist()[0] 
    data_open = data[market_open_time:market_close_time+1]
    timestamp_ = data_open.TimeStamp.unique()
    return timestamp_

In [30]:
def time_transform(timestamp_time):
    time_second = []
    for i in range(0,len(timestamp_time),1):
        second = float(timestamp_time[i][11])*36000 + float(timestamp_time[i][12])*3600 \
                +float(timestamp_time[i][14])*600 + float(timestamp_time[i][15])*60\
                +float(timestamp_time[i][17])*10 + float(timestamp_time[i][18])  
        time_second.append(second - 32400.0)
    return  time_second

In [31]:
def bid123_ask123_Q(data_book_28_open):
    Bid1 = []
    Bid2 = []
    Bid3 = []
    Bid1_Quantity = []
    Bid2_Quantity = []
    Bid3_Quantity = []
    Ask1 = []
    Ask2 = []
    Ask3 = []
    Ask1_Quantity = []
    Ask2_Quantity = []
    Ask3_Quantity = []

    TimeStamp = []
    for i in range(1,len(data_book_28_open),4):
        #print data_book_28_open.iloc[i]['0']
        #print data_book_28_open.iloc[i]['2']
        Bid1.append(float(data_book_28_open.iloc[i]['0'])/100.0)
        Bid1_Quantity.append(float(data_book_28_open.iloc[i]['1']))

        Bid2.append(float(data_book_28_open.iloc[i + 1]['0'])/100.0)
        Bid2_Quantity.append(float(data_book_28_open.iloc[i + 1]['1']))  

        Bid3.append(float(data_book_28_open.iloc[i + 2]['0'])/100.0)
        Bid3_Quantity.append(float(data_book_28_open.iloc[i + 2]['1']))  

        Ask1.append(float(data_book_28_open.iloc[i]['2'])/100.0) 
        Ask1_Quantity.append(float(data_book_28_open.iloc[i]['3']))

        Ask2.append(float(data_book_28_open.iloc[i + 1]['2'])/100.0) 
        Ask2_Quantity.append(float(data_book_28_open.iloc[i + 1]['3']))    

        Ask3.append(float(data_book_28_open.iloc[i + 2]['2'])/100.0) 
        Ask3_Quantity.append(float(data_book_28_open.iloc[i + 2]['3']))    
        TimeStamp.append(data_book_28_open.iloc[i-1][1])
    return Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity,    

In [32]:
def rise_bid(Bid1,timestamp_time_second):
    rise_ratio = []
    index = np.where(np.array(timestamp_time_second) >= 600)[0][0]
    for i in range(0,index):
        rise_ratio_ = round((Bid1[i] - Bid1[0])*(1.0)/Bid1[0]*100,5)
        rise_ratio.append(rise_ratio_)
    for i in range(index,len(Bid1),1):
        index_start = np.where(np.array(timestamp_time_second[:i]) >= timestamp_time_second[i] - 600)[0][0]
        rise_ratio_ = round((Bid1[i] - Bid1[index_start])*(1.0)/Bid1[index_start]*100,5)
        rise_ratio.append(rise_ratio_)
    return rise_ratio

In [33]:
def rise_cal(i,rise_ratio_bid,timestamp_time_second,Bid,Ask,index,index_15_min,close_price,\
             N_1,N_2,N_3,N_4,N_5,N_1_T,N_2_T,N_3_T,N_4_T,N_5_T,\
             N_1_E,N_2_E,N_3_E,N_4_E,N_5_E,N_1_M,N_2_M,N_3_M,N_4_M,N_5_M,\
             N_1_TI,N_2_TI,N_3_TI,N_4_TI,N_5_TI):
    
    if rise_ratio_bid[index] <= -0.4:
        N_1 = N_1 + 1
        #print rise_ratio_ask[index]
        #Ask1[i] < max_Bid1_time_series[i+1]
        if Ask[index] < max(Bid[(index+1):index + index_15_min + 1]):
            N_1_T = N_1_T + 1
            N_1_E = N_1_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_1_E
            first_max_index = np.where(np.array(Bid[(index + 1):index + 1 + index_15_min + 1 ]) > Ask[index])[0][0]
            N_1_M = N_1_M + (Ask[index] - min(Bid[(index + 1):(index + 1 + first_max_index + 1)]))*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_max_index + 1]
            N_1_TI = N_1_TI + (time_traded - time_start)
            #print N_1_TI
            pass
        else:
            if i < 25200 - 900:
                N_1_E = N_1_E + (Bid[index + index_15_min] - Bid[index])*(1.0)/(5.0)
                N_1_M = N_1_M + (Ask[index] - min(Bid[(index+1):(index + 1 + index_15_min + 1 )]))*(1.0)/(5.0) 
                #print N_1_M
                N_1_TI = N_1_TI + (900.0)
                #print N_1_TI
            else:
                N_1_E = N_1_E + (close_price - Bid[index])*(1.0)/(5.0)
                N_1_M = N_1_M + (Ask[index] - min(Bid[(index+1):]))*(1.0)/(5.0) 
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_1_TI = N_1_TI + (time_traded - time_start)
                #print N_1_TI
            pass

    elif -0.4 < rise_ratio_bid[index] <= -0.15:
        N_2 = N_2 + 1
        #print rise_ratio_ask[index]
        if Ask[index] < max(Bid[(index+1):index + index_15_min + 1]):
            #print 3
            N_2_T = N_2_T + 1
            N_2_E = N_2_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_2_E
            first_max_index = np.where(np.array(Bid[(index+1):index + 1 + index_15_min + 1 ]) > Ask[index])[0][0]
            N_2_M = N_2_M + (Ask[index] - min(Bid[(index + 1):(index + 1 + first_max_index + 1)]))*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_max_index + 1]
            N_2_TI = N_2_TI + (time_traded - time_start)
            #print N_2_TI
            pass
        else:
            if i < 25200 - 900:
                N_2_E = N_2_E + (Bid[index + index_15_min] - Bid[index])*(1.0)/(5.0)
                N_2_M = N_2_M + (Ask[index] - min(Bid[(index+1):(index + 1 + index_15_min + 1 )]))*(1.0)/(5.0) 
                N_2_TI = N_2_TI + (900.0)
                #print N_2_TI
            else:
                N_2_E = N_2_E + (close_price - Bid[index])*(1.0)/(5.0)
                N_2_M = N_2_M + (Ask[index] - min(Bid[(index+1):]))*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_2_TI = N_2_TI + (time_traded - time_start)
                #print N_2_TI
            pass

    elif -0.15 < rise_ratio_bid[index] <= 0.15:
        N_3 = N_3 + 1
        #print rise_ratio_ask[index]
        if Ask[index] < max(Bid[(index+1):index + index_15_min + 1]):
            #print 5
            N_3_T = N_3_T + 1
            N_3_E = N_3_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_3_E
            first_max_index = np.where(np.array(Bid[(index + 1):index + 1 + index_15_min + 1]) > Ask[index])[0][0]
            N_3_M = N_3_M + (Ask[index] - min(Bid[(index + 1):(index + 1 + first_max_index + 1)]))*(1.0)/(5.0) 
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_max_index + 1]
            N_3_TI = N_3_TI + (time_traded - time_start)
            #print N_3_TI
            pass
        else:
            if i < 25200 - 900:
                N_3_E = N_3_E + (Bid[index + index_15_min] - Bid[index])*(1.0)/(5.0)
                N_3_M = N_3_M + (Ask[index] - min(Bid[(index+1):(index + 1 + index_15_min + 1 )]))*(1.0)/(5.0) 
                N_3_TI = N_3_TI + (900.0)
                #print N_3_TI
            else:
                N_3_E = N_3_E + (close_price - Bid[index])*(1.0)/(5.0)
                N_3_M = N_3_M + (Ask[index] - min(Bid[(index+1):]))*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_3_TI = N_3_TI + (time_traded - time_start)
                #print N_3_TI
            pass

    elif 0.15 < rise_ratio_bid[index] < 0.4:
        N_4 = N_4 + 1
        #print rise_ratio_ask[index]
        if Ask[index] < max(Bid[(index+1):index + index_15_min + 1]):
            #print 7
            N_4_T = N_4_T + 1
            N_4_E = N_4_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_4_E
            first_max_index = np.where(np.array(Bid[(index+1):index + 1 + index_15_min + 1 ]) > Ask[index])[0][0]
            N_4_M = N_4_M + (Ask[index] - min(Bid[(index + 1):(index + 1 + first_max_index + 1)]))*(1.0)/(5.0) 
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_max_index + 1]
            N_4_TI = N_4_TI + (time_traded - time_start)
            #print N_4_TI
            pass
        else:
            if i < 25200 - 900:
                N_4_E = N_4_E + (Bid[index + index_15_min] - Bid[index])*(1.0)/(5.0)
                N_4_M = N_4_M + (Ask[index] - min(Bid[(index+1):(index + 1 + index_15_min + 1 )]))*(1.0)/(5.0) 
                N_4_TI = N_4_TI + (900.0)
                #print N_4_TI
            else:
                N_4_E = N_4_E + (close_price - Bid[index])*(1.0)/(5.0)
                N_4_M = N_4_M + (Ask[index] - min(Bid[(index+1):]))*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_4_TI = N_4_TI + (time_traded - time_start)
                #print N_4_TI
            pass

    elif 0.4 <= rise_ratio_bid[index]:
        N_5 = N_5 + 1
        #print rise_ratio_ask[index]
        if Ask[index] < max(Bid[(index+1):index + index_15_min + 1]):
            N_5_T = N_5_T + 1
            N_5_E = N_5_E + (Ask[index] - Bid[index])*(1.0)/(5.0)
            #print N_5_E      
            first_max_index = np.where(np.array(Bid[(index+1):index + 1 + index_15_min + 1 ]) > Ask[index])[0][0]
            N_5_M = N_5_M + (Ask[index] - min(Bid[(index + 1):(index + 1 + first_max_index + 1)]))*(1.0)/(5.0) 
            
            time_start = timestamp_time_second[index]
            time_traded = timestamp_time_second[index + first_max_index + 1]
            N_5_TI = N_5_TI + (time_traded - time_start)
            #print N_5_TI
            pass
        else:
            if i < 25200 - 900:
                N_5_E = N_5_E + (Bid[index + index_15_min] - Bid[index])*(1.0)/(5.0)
                N_5_M = N_5_M + (Ask[index] - min(Bid[(index+1):(index + 1 + index_15_min + 1)]))*(1.0)/(5.0) 
                N_1_TI = N_1_TI + (900.0)
                #print N_5_TI
            else:
                N_5_E = N_5_E + (close_price - Bid[index])*(1.0)/(5.0)
                N_5_M = N_5_M + (Ask[index] - min(Bid[(index+1):]))*(1.0)/(5.0)
                time_start = timestamp_time_second[index]
                time_traded = timestamp_time_second[len(timestamp_time_second)-1]                
                N_5_TI = N_5_TI + (time_traded - time_start)
                #print N_5_TI 
            pass
        
    return N_1,N_2,N_3,N_4,N_5,N_1_T,N_2_T,N_3_T,N_4_T,N_5_T,N_1_E,N_2_E,N_3_E,N_4_E,N_5_E,N_1_M,N_2_M,N_3_M,N_4_M,N_5_M,\
           N_1_TI,N_2_TI,N_3_TI,N_4_TI,N_5_TI

In [34]:
def ask(time1,time2,Bid,Ask,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,\
        Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weigth1,weigth2,weigth3):
    [N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,\
     N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI] = [0 for i in  range(0,25,1)]
    [N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,\
     N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI] = [0 for i in  range(0,25,1)]
    [N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,\
     N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI] = [0 for i in  range(0,25,1)]
    [N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,\
     N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI] = [0 for i in  range(0,25,1)]
    [N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,\
     N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI] = [0 for i in  range(0,25,1)]

    for i in range(time1,time2,1):
        index = np.where(array(timestamp_time_second) <= i)[0][-1]
        if i < 25200 - 900:
            index_15_min = np.where(np.array(timestamp_time_second[index:]) >= i + 900)[0][0]

        weighted_Bid = Bid1_Quantity[index] * weight1 + Bid2_Quantity[index] * weight2 + Bid3_Quantity[index] * weight3
        weighted_Ask = Ask1_Quantity[index] * weight1 + Ask2_Quantity[index] * weight2 + Ask3_Quantity[index] * weight3  
        if (weighted_Bid*(1.0) / weighted_Ask) <= 0.7:
            N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,\
            N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI = rise_cal(i,rise_ratio_bid,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_11,N_12,N_13,N_14,N_15,N_11_T,N_12_T,N_13_T,N_14_T,N_15_T,\
                                                          N_11_E,N_12_E,N_13_E,N_14_E,N_15_E,N_11_M,N_12_M,N_13_M,N_14_M,N_15_M,\
                                                          N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI)                                               
        elif 0.7 < (weighted_Bid*(1.0) / weighted_Ask) <= 0.9:
            N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,\
            N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI = rise_cal(i,rise_ratio_bid,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_21,N_22,N_23,N_24,N_25,N_21_T,N_22_T,N_23_T,N_24_T,N_25_T,\
                                                          N_21_E,N_22_E,N_23_E,N_24_E,N_25_E,N_21_M,N_22_M,N_23_M,N_24_M,N_25_M,\
                                                          N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI)                                                
        elif 0.9 < (weighted_Bid*(1.0) / weighted_Ask) <= 1.1:
            N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,\
            N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI = rise_cal(i,rise_ratio_bid,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_31,N_32,N_33,N_34,N_35,N_31_T,N_32_T,N_33_T,N_34_T,N_35_T,\
                                                          N_31_E,N_32_E,N_33_E,N_34_E,N_35_E,N_31_M,N_32_M,N_33_M,N_34_M,N_35_M,\
                                                          N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI)                                    
        elif 1.1 < (weighted_Bid*(1.0) / weighted_Ask) <= 1.3:
            N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,\
            N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI = rise_cal(i,rise_ratio_bid,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_41,N_42,N_43,N_44,N_45,N_41_T,N_42_T,N_43_T,N_44_T,N_45_T,\
                                                          N_41_E,N_42_E,N_43_E,N_44_E,N_45_E,N_41_M,N_42_M,N_43_M,N_44_M,N_45_M,\
                                                          N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI)                              
        elif 1.30 < (weighted_Bid*(1.0) / weighted_Ask):
            N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,\
            N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI = rise_cal(i,rise_ratio_bid,timestamp_time_second,Bid,Ask,index,index_15_min,\
                                                          close_price,N_51,N_52,N_53,N_54,N_55,N_51_T,N_52_T,N_53_T,N_54_T,N_55_T,\
                                                          N_51_E,N_52_E,N_53_E,N_54_E,N_55_E,N_51_M,N_52_M,N_53_M,N_54_M,N_55_M,\
                                                          N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI)                 
    return [[N_11,N_12,N_13,N_14,N_15],[N_11_T,N_12_T,N_13_T,N_14_T,N_15_T],[N_11_E,N_12_E,N_13_E,N_14_E,N_15_E],\
             [N_11_M,N_12_M,N_13_M,N_14_M,N_15_M],[N_11_TI,N_12_TI,N_13_TI,N_14_TI,N_15_TI],\
            [N_21,N_22,N_23,N_24,N_25],[N_21_T,N_22_T,N_23_T,N_24_T,N_25_T],[N_21_E,N_22_E,N_23_E,N_24_E,N_25_E],\
             [N_21_M,N_22_M,N_23_M,N_24_M,N_25_M],[N_21_TI,N_22_TI,N_23_TI,N_24_TI,N_25_TI],\
            [N_31,N_32,N_33,N_34,N_35],[N_31_T,N_32_T,N_33_T,N_34_T,N_35_T],[N_31_E,N_32_E,N_33_E,N_34_E,N_35_E],\
             [N_31_M,N_32_M,N_33_M,N_34_M,N_35_M],[N_31_TI,N_32_TI,N_33_TI,N_34_TI,N_35_TI],\
            [N_41,N_42,N_43,N_44,N_45],[N_41_T,N_42_T,N_43_T,N_44_T,N_45_T],[N_41_E,N_42_E,N_43_E,N_44_E,N_45_E],\
             [N_41_M,N_42_M,N_43_M,N_44_M,N_45_M],[N_41_TI,N_42_TI,N_43_TI,N_44_TI,N_45_TI],\
            [N_51,N_52,N_53,N_54,N_55],[N_51_T,N_52_T,N_53_T,N_54_T,N_55_T],[N_51_E,N_52_E,N_53_E,N_54_E,N_55_E],\
             [N_51_M,N_52_M,N_53_M,N_54_M,N_55_M],[N_51_TI,N_52_TI,N_53_TI,N_54_TI,N_55_TI]] 




### Day 2/07

In [35]:
data_book = order_book(2,'18')
data_book_open = data_book[1028:301211+1]
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,'18')
timestamp_time_second = time_transform(timestamp_time)

In [36]:
#data_book_open

In [37]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 113.567939


In [38]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 88.695432


In [39]:
close_price = 6809
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [40]:
time1 = 0
time2 = 600
mat28_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [41]:
#mat28_3_15 

In [42]:
time1 = 600
time2 = 7200
mat28_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [43]:
#mat28_3_11

In [44]:
time1 = 7200
time2 = 14400
mat28_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [45]:
time1 = 14400
time2 = 25200
mat28_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat28_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [46]:
#mat28_3_16

### Day 2/19

In [47]:
data_book = order_book(2,19)
data_book_open = data_book[1152:344127+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,19)
timestamp_time_second = time_transform(timestamp_time)

In [48]:
#data_book_open 

In [49]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 117.987819


In [50]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 119.904182


In [51]:
close_price = 6941
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [52]:
time1 = 0
time2 = 600
mat29_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [53]:
#mat29_3_15

In [54]:
time1 = 600
time2 = 7200
mat29_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [55]:
time1 = 7200
time2 = 14400
mat29_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [56]:
time1 = 14400
time2 = 25200
mat29_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat29_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [57]:
#mat29_2_13

### Day 2/24

In [58]:
data_book = order_book(2,'24')
data_book_open = data_book[660:438423+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,'24')
timestamp_time_second = time_transform(timestamp_time)

In [59]:
#data_book_open

In [60]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 149.386833


In [61]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 234.051818


In [62]:
close_price = 6625
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [63]:
time1 = 0
time2 = 600
mat27_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [64]:
time1 = 600
time2 = 7200
mat27_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [65]:
time1 = 7200
time2 = 14400
mat27_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [66]:
time1 = 14400
time2 = 25200
mat27_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat27_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [67]:
#mat27_2_16

### Day 2/25

In [68]:
data_book = order_book(2,25)
data_book_open = data_book[1028:380387+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,25)
timestamp_time_second = time_transform(timestamp_time)

In [69]:
#data_book_open

In [70]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 130.017340


In [71]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 324.544566


In [72]:
close_price = 6497
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [73]:
time1 = 0
time2 = 600
mat24_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [74]:
time1 = 600
time2 = 7200
mat24_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [75]:
time1 = 7200
time2 = 14400
mat24_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [76]:
time1 = 14400
time2 = 25200
mat24_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat24_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 2/26

In [77]:
data_book = order_book(2,26)
data_book_open = data_book[892:312443+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,26)
timestamp_time_second = time_transform(timestamp_time)

In [78]:
#data_book_open

In [79]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 105.906990


In [80]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 211.285702


In [81]:
close_price = 6503
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [82]:
time1 = 0
time2 = 600
mat22_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [83]:
time1 = 600
time2 = 7200
mat22_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [84]:
time1 = 7200
time2 = 14400
mat22_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [85]:
time1 = 14400
time2 = 25200
mat22_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat22_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [86]:
#mat22_3_15

### Day 2/27

In [87]:
data_book = order_book(2,27)
data_book_open = data_book[444:212171+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,27)
timestamp_time_second = time_transform(timestamp_time)

In [88]:
#data_book_open

In [89]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 72.087187


In [90]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 120.179301


In [91]:
close_price = 6535
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [92]:
time1 = 0
time2 = 600
mat21_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [93]:
time1 = 600
time2 = 7200
mat21_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [94]:
time1 = 7200
time2 = 14400
mat21_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [95]:
time1 = 14400
time2 = 25200
mat21_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat21_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

### Day 2/17

In [96]:
data_book = order_book(2,17)
data_book_open = data_book[668:299679+1] # 9:00 ~ 16:00
data_book_open = data_book_open.reset_index(drop = True)
timestamp_time = day_time(2,17)
timestamp_time_second = time_transform(timestamp_time)

In [97]:
#data_book_open

In [98]:
import time 
start = time.time()
Bid1,Bid1_Quantity,Bid2,Bid2_Quantity,Bid3,Bid3_Quantity,Ask1,Ask1_Quantity,Ask2,Ask2_Quantity,Ask3,Ask3_Quantity = bid123_ask123_Q(data_book_open)
end = time.time()  
print "Total time = %f"%(end - start)

Total time = 102.166364


In [99]:
import time 
start = time.time()
rise_ratio_bid = rise_bid(Bid1,timestamp_time_second)
end = time.time()  
print "Total time = %f"%(end - start) 

Total time = 261.611485


In [100]:
close_price = 6936
weight1 = 0.50
weight2 = 0.30
weight3 = 0.20

In [101]:
time1 = 0
time2 = 600
mat20_1_15 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_15 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_15 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [102]:
time1 = 600
time2 = 7200
mat20_1_11 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_11 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_11 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [103]:
time1 = 7200
time2 = 14400
mat20_1_13 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_13 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_13 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [104]:
time1 = 14400
time2 = 25200
mat20_1_16 = ask(time1,time2,Bid1,Ask1,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_2_16 = ask(time1,time2,Bid1,Ask2,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)
mat20_3_16 = ask(time1,time2,Bid1,Ask3,close_price,timestamp_time_second,Bid1_Quantity,Bid2_Quantity,Bid3_Quantity,Ask1_Quantity,Ask2_Quantity,Ask3_Quantity,weight1,weight2,weight3)

In [105]:
#mat20_2_15

In [106]:
SUM_Ask1_15 = \
array(mat20_1_15)+array(mat21_1_15)+array(mat22_1_15)+\
array(mat24_1_15)+array(mat27_1_15)+array(mat28_1_15)+array(mat29_1_15)
SUM_Ask2_15 = \
array(mat20_2_15)+array(mat21_2_15)+array(mat22_2_15)+\
array(mat24_2_15)+array(mat27_2_15)+array(mat28_2_15)+array(mat29_2_15)
SUM_Ask3_15 = \
array(mat20_3_15)+array(mat21_3_15)+array(mat22_3_15)+\
array(mat24_3_15)+array(mat27_3_15)+array(mat28_3_15)+array(mat29_3_15)

In [107]:
#SUM_Bid3_15

In [108]:
SUM_Ask1_11 = \
array(mat20_1_11)+array(mat21_1_11)+array(mat22_1_11)+\
array(mat24_1_11)+array(mat27_1_11)+array(mat28_1_11)+array(mat29_1_11)
SUM_Ask2_11 = \
array(mat20_2_11)+array(mat21_2_11)+array(mat22_2_11)+\
array(mat24_2_11)+array(mat27_2_11)+array(mat28_2_11)+array(mat29_2_11)
SUM_Ask3_11 = \
+array(mat20_3_11)+array(mat21_3_11)+array(mat22_3_11)+\
array(mat24_3_11)+array(mat27_3_11)+array(mat28_3_11)+array(mat29_3_11)

In [109]:
#SUM_Ask3_11

In [110]:
SUM_Ask1_13 = \
array(mat20_1_13)+array(mat21_1_13)+array(mat22_1_13)+\
array(mat24_1_13)+array(mat27_1_13)+array(mat28_1_13)+array(mat29_1_13)
SUM_Ask2_13 = \
array(mat20_2_13)+array(mat21_2_13)+array(mat22_2_13)+\
array(mat24_2_13)+array(mat27_2_13)+array(mat28_2_13)+array(mat29_2_13)
SUM_Ask3_13 = \
array(mat20_3_13)+array(mat21_3_13)+array(mat22_3_13)+\
array(mat24_3_13)+array(mat27_3_13)+array(mat28_3_13)+array(mat29_3_13)

In [111]:
#SUM_Bid3_13

In [112]:
SUM_Ask1_16 = \
array(mat20_1_16)+array(mat21_1_16)+array(mat22_1_16)+\
array(mat24_1_16)+array(mat27_1_16)+array(mat28_1_16)+array(mat29_1_16)
SUM_Ask2_16 = \
array(mat20_2_16)+array(mat21_2_16)+array(mat22_2_16)+\
array(mat24_2_16)+array(mat27_2_16)+array(mat28_2_16)+array(mat29_2_16)
SUM_Ask3_16 = \
array(mat20_3_16)+array(mat21_3_16)+array(mat22_3_16)+\
array(mat24_3_16)+array(mat27_3_16)+array(mat28_3_16)+array(mat29_3_16)

In [113]:
#SUM_Bid3_16

In [114]:
a = 0
Ask1_15_1 = pd.DataFrame([SUM_Ask1_15[a],SUM_Ask1_15[a+1]/SUM_Ask1_15[a],SUM_Ask1_15[a+2]/SUM_Ask1_15[a],SUM_Ask1_15[a+3]/SUM_Ask1_15[a],SUM_Ask1_15[a+4]/SUM_Ask1_15[a]])

In [115]:
a = 5
Ask1_15_2 = pd.DataFrame([SUM_Ask1_15[a],SUM_Ask1_15[a+1]/SUM_Ask1_15[a],SUM_Ask1_15[a+2]/SUM_Ask1_15[a],SUM_Ask1_15[a+3]/SUM_Ask1_15[a],SUM_Ask1_15[a+4]/SUM_Ask1_15[a]])

In [116]:
a = 10
Ask1_15_3 = pd.DataFrame([SUM_Ask1_15[a],SUM_Ask1_15[a+1]/SUM_Ask1_15[a],SUM_Ask1_15[a+2]/SUM_Ask1_15[a],SUM_Ask1_15[a+3]/SUM_Ask1_15[a],SUM_Ask1_15[a+4]/SUM_Ask1_15[a]])

In [117]:
a = 15
Ask1_15_4 = pd.DataFrame([SUM_Ask1_15[a],SUM_Ask1_15[a+1]/SUM_Ask1_15[a],SUM_Ask1_15[a+2]/SUM_Ask1_15[a],SUM_Ask1_15[a+3]/SUM_Ask1_15[a],SUM_Ask1_15[a+4]/SUM_Ask1_15[a]])

In [118]:
a = 20
Ask1_15_5 = pd.DataFrame([SUM_Ask1_15[a],SUM_Ask1_15[a+1]/SUM_Ask1_15[a],SUM_Ask1_15[a+2]/SUM_Ask1_15[a],SUM_Ask1_15[a+3]/SUM_Ask1_15[a],SUM_Ask1_15[a+4]/SUM_Ask1_15[a]])

In [119]:
pd.concat([Ask1_15_1,Ask1_15_2,Ask1_15_3,Ask1_15_4,Ask1_15_5])

,0,1,2,3,4
0,0.0,338.000000,1177.000000,503.000000,0.0
1,NaN,0.997041,0.424809,0.564612,NaN
2,NaN,2.600592,-4.823280,-0.502982,NaN
3,NaN,3.233728,7.564996,3.351889,NaN
4,NaN,502.863905,737.293968,736.113320,NaN
0,0.0,0.000000,274.000000,247.000000,0.0
1,NaN,NaN,0.270073,0.785425,NaN
2,NaN,NaN,-1.708029,0.748988,NaN
3,NaN,NaN,5.521898,1.870445,NaN
4,NaN,NaN,800.645985,598.943320,NaN


In [120]:
pd.concat([Ask1_15_1,Ask1_15_2,Ask1_15_3,Ask1_15_4,Ask1_15_5]).to_csv('ask1_15_218_227.csv')

In [122]:
#218_227218_227

In [123]:
a = 0
Ask2_15_1 = pd.DataFrame([SUM_Ask2_15[a],SUM_Ask2_15[a+1]/SUM_Ask2_15[a],SUM_Ask2_15[a+2]/SUM_Ask2_15[a],SUM_Ask2_15[a+3]/SUM_Ask2_15[a],SUM_Ask2_15[a+4]/SUM_Ask2_15[a]])

In [124]:
a = 5
Ask2_15_2 = pd.DataFrame([SUM_Ask2_15[a],SUM_Ask2_15[a+1]/SUM_Ask2_15[a],SUM_Ask2_15[a+2]/SUM_Ask2_15[a],SUM_Ask2_15[a+3]/SUM_Ask2_15[a],SUM_Ask2_15[a+4]/SUM_Ask2_15[a]])

In [125]:
a = 10
Ask2_15_3 = pd.DataFrame([SUM_Ask2_15[a],SUM_Ask2_15[a+1]/SUM_Ask2_15[a],SUM_Ask2_15[a+2]/SUM_Ask2_15[a],SUM_Ask2_15[a+3]/SUM_Ask2_15[a],SUM_Ask2_15[a+4]/SUM_Ask2_15[a]])

In [126]:
a = 15
Ask2_15_4 = pd.DataFrame([SUM_Ask2_15[a],SUM_Ask2_15[a+1]/SUM_Ask2_15[a],SUM_Ask2_15[a+2]/SUM_Ask2_15[a],SUM_Ask2_15[a+3]/SUM_Ask2_15[a],SUM_Ask2_15[a+4]/SUM_Ask2_15[a]])

In [127]:
a = 20
Ask2_15_5 = pd.DataFrame([SUM_Ask2_15[a],SUM_Ask2_15[a+1]/SUM_Ask2_15[a],SUM_Ask2_15[a+2]/SUM_Ask2_15[a],SUM_Ask2_15[a+3]/SUM_Ask2_15[a],SUM_Ask2_15[a+4]/SUM_Ask2_15[a]])

In [128]:
pd.concat([Ask2_15_1,Ask2_15_2,Ask2_15_3,Ask2_15_4,Ask2_15_5])

,0,1,2,3,4
0,0.0,338.000000,1177.000000,503.000000,0.0
1,NaN,0.994083,0.412914,0.453280,NaN
2,NaN,3.597633,-4.414613,-0.375746,NaN
3,NaN,4.233728,8.644010,4.886680,NaN
4,NaN,656.642012,772.828377,780.679920,NaN
0,0.0,0.000000,274.000000,247.000000,0.0
1,NaN,NaN,0.262774,0.340081,NaN
2,NaN,NaN,-1.452555,-0.214575,NaN
3,NaN,NaN,6.777372,5.048583,NaN
4,NaN,NaN,862.222628,818.368421,NaN


In [130]:
pd.concat([Ask2_15_1,Ask2_15_2,Ask2_15_3,Ask2_15_4,Ask2_15_5]).to_csv('ask2_15_218_227.csv')

In [131]:
a = 0
Ask3_15_1 = pd.DataFrame([SUM_Ask3_15[a],SUM_Ask3_15[a+1]/SUM_Ask3_15[a],SUM_Ask3_15[a+2]/SUM_Ask3_15[a],SUM_Ask3_15[a+3]/SUM_Ask3_15[a],SUM_Ask3_15[a+4]/SUM_Ask3_15[a]])

In [132]:
a = 5
Ask3_15_2 = pd.DataFrame([SUM_Ask3_15[a],SUM_Ask3_15[a+1]/SUM_Ask3_15[a],SUM_Ask3_15[a+2]/SUM_Ask3_15[a],SUM_Ask3_15[a+3]/SUM_Ask3_15[a],SUM_Ask3_15[a+4]/SUM_Ask3_15[a]])

In [133]:
a = 10
Ask3_15_3 = pd.DataFrame([SUM_Ask3_15[a],SUM_Ask3_15[a+1]/SUM_Ask3_15[a],SUM_Ask3_15[a+2]/SUM_Ask3_15[a],SUM_Ask3_15[a+3]/SUM_Ask3_15[a],SUM_Ask3_15[a+4]/SUM_Ask3_15[a]])

In [134]:
a = 15
Ask3_15_4 = pd.DataFrame([SUM_Ask3_15[a],SUM_Ask3_15[a+1]/SUM_Ask3_15[a],SUM_Ask3_15[a+2]/SUM_Ask3_15[a],SUM_Ask3_15[a+3]/SUM_Ask3_15[a],SUM_Ask3_15[a+4]/SUM_Ask3_15[a]])

In [135]:
a = 20
Ask3_15_5 = pd.DataFrame([SUM_Ask3_15[a],SUM_Ask3_15[a+1]/SUM_Ask3_15[a],SUM_Ask3_15[a+2]/SUM_Ask3_15[a],SUM_Ask3_15[a+3]/SUM_Ask3_15[a],SUM_Ask3_15[a+4]/SUM_Ask3_15[a]])

In [136]:
pd.concat([Ask3_15_1,Ask3_15_2,Ask3_15_3,Ask3_15_4,Ask3_15_5])

,0,1,2,3,4
0,0.0,338.000000,1177.000000,503.000000,0.0
1,NaN,0.994083,0.201359,0.445328,NaN
2,NaN,4.591716,-4.588785,0.073559,NaN
3,NaN,5.233728,9.644010,5.886680,NaN
4,NaN,707.177515,841.606627,781.771372,NaN
0,0.0,0.000000,274.000000,247.000000,0.0
1,NaN,NaN,0.014599,0.340081,NaN
2,NaN,NaN,-1.890511,0.125506,NaN
3,NaN,NaN,7.777372,6.048583,NaN
4,NaN,NaN,897.755474,819.048583,NaN


In [137]:
pd.concat([Ask3_15_1,Ask3_15_2,Ask3_15_3,Ask3_15_4,Ask3_15_5]).to_csv('ask3_15_218_227.csv')

In [138]:
a = 0
Ask1_11_1 = pd.DataFrame([SUM_Ask1_11[a],SUM_Ask1_11[a+1]/SUM_Ask1_11[a],SUM_Ask1_11[a+2]/SUM_Ask1_11[a],SUM_Ask1_11[a+3]/SUM_Ask1_11[a],SUM_Ask1_11[a+4]/SUM_Ask1_11[a]])

In [139]:
a = 5
Ask1_11_2 = pd.DataFrame([SUM_Ask1_11[a],SUM_Ask1_11[a+1]/SUM_Ask1_11[a],SUM_Ask1_11[a+2]/SUM_Ask1_11[a],SUM_Ask1_11[a+3]/SUM_Ask1_11[a],SUM_Ask1_11[a+4]/SUM_Ask1_11[a]])

In [140]:
a = 10
Ask1_11_3 = pd.DataFrame([SUM_Ask1_11[a],SUM_Ask1_11[a+1]/SUM_Ask1_11[a],SUM_Ask1_11[a+2]/SUM_Ask1_11[a],SUM_Ask1_11[a+3]/SUM_Ask1_11[a],SUM_Ask1_11[a+4]/SUM_Ask1_11[a]])

In [141]:
a = 15
Ask1_11_4 = pd.DataFrame([SUM_Ask1_11[a],SUM_Ask1_11[a+1]/SUM_Ask1_11[a],SUM_Ask1_11[a+2]/SUM_Ask1_11[a],SUM_Ask1_11[a+3]/SUM_Ask1_11[a],SUM_Ask1_11[a+4]/SUM_Ask1_11[a]])

In [142]:
a = 20
Ask1_11_5 = pd.DataFrame([SUM_Ask1_11[a],SUM_Ask1_11[a+1]/SUM_Ask1_11[a],SUM_Ask1_11[a+2]/SUM_Ask1_11[a],SUM_Ask1_11[a+3]/SUM_Ask1_11[a],SUM_Ask1_11[a+4]/SUM_Ask1_11[a]])

In [143]:
pd.concat([Ask1_11_1,Ask1_11_2,Ask1_11_3,Ask1_11_4,Ask1_11_5])

,0,1,2,3,4
0,375.000000,1187.000000,4205.000000,872.000000,435.000000
1,0.357333,0.396799,0.372176,0.378440,0.577011
2,-3.173333,-2.165965,-2.074911,-1.511468,-1.590805
3,5.938667,5.312553,4.770749,4.529817,4.308046
4,1106.029333,685.037911,681.668252,654.873853,131.563218
0,546.000000,2056.000000,4919.000000,1119.000000,392.000000
1,0.441392,0.447471,0.435861,0.461126,0.625000
2,-2.717949,-1.302529,-1.248018,-1.151028,-0.959184
3,5.736264,4.472763,4.048790,4.118856,4.099490
4,851.712454,628.062743,658.554788,592.650581,163.265306


In [144]:
pd.concat([Ask1_11_1,Ask1_11_2,Ask1_11_3,Ask1_11_4,Ask1_11_5]).to_csv('ask1_11_218_227.csv')

In [145]:
a = 0
Ask2_11_1 = pd.DataFrame([SUM_Ask2_11[a],SUM_Ask2_11[a+1]/SUM_Ask2_11[a],SUM_Ask2_11[a+2]/SUM_Ask2_11[a],SUM_Ask2_11[a+3]/SUM_Ask2_11[a],SUM_Ask2_11[a+4]/SUM_Ask2_11[a]])

In [146]:
a = 5
Ask2_11_2 = pd.DataFrame([SUM_Ask2_11[a],SUM_Ask2_11[a+1]/SUM_Ask2_11[a],SUM_Ask2_11[a+2]/SUM_Ask2_11[a],SUM_Ask2_11[a+3]/SUM_Ask2_11[a],SUM_Ask2_11[a+4]/SUM_Ask2_11[a]])

In [147]:
a = 10
Ask2_11_3 = pd.DataFrame([SUM_Ask2_11[a],SUM_Ask2_11[a+1]/SUM_Ask2_11[a],SUM_Ask2_11[a+2]/SUM_Ask2_11[a],SUM_Ask2_11[a+3]/SUM_Ask2_11[a],SUM_Ask2_11[a+4]/SUM_Ask2_11[a]])

In [148]:
a = 15
Ask2_11_4 = pd.DataFrame([SUM_Ask2_11[a],SUM_Ask2_11[a+1]/SUM_Ask2_11[a],SUM_Ask2_11[a+2]/SUM_Ask2_11[a],SUM_Ask2_11[a+3]/SUM_Ask2_11[a],SUM_Ask2_11[a+4]/SUM_Ask2_11[a]])

In [149]:
a = 20
Ask2_11_5 = pd.DataFrame([SUM_Ask2_11[a],SUM_Ask2_11[a+1]/SUM_Ask2_11[a],SUM_Ask2_11[a+2]/SUM_Ask2_11[a],SUM_Ask2_11[a+3]/SUM_Ask2_11[a],SUM_Ask2_11[a+4]/SUM_Ask2_11[a]])

In [150]:
pd.concat([Ask2_11_1,Ask2_11_2,Ask2_11_3,Ask2_11_4,Ask2_11_5])

,0,1,2,3,4
0,375.000000,1187.000000,4205.000000,872.000000,435.000000
1,0.216000,0.292334,0.200476,0.191514,0.310345
2,-3.296000,-2.005897,-2.028537,-1.863532,-3.131034
3,7.330667,6.385847,5.891558,5.876147,6.834483
4,1515.570667,762.195451,790.061831,781.363532,78.779310
0,546.000000,2056.000000,4919.000000,1119.000000,392.000000
1,0.305861,0.324903,0.243545,0.262735,0.331633
2,-2.706960,-1.190175,-1.152470,-1.393208,-2.673469
3,7.089744,5.615759,5.197195,5.468275,6.892857
4,1177.879121,743.144942,781.105103,739.086685,102.627551


In [152]:
pd.concat([Ask2_11_1,Ask2_11_2,Ask2_11_3,Ask2_11_4,Ask2_11_5]).to_csv('ask2_11_218_227.csv')

In [153]:
a = 0
Ask3_11_1 = pd.DataFrame([SUM_Ask3_11[a],SUM_Ask3_11[a+1]/SUM_Ask3_11[a],SUM_Ask3_11[a+2]/SUM_Ask3_11[a],SUM_Ask3_11[a+3]/SUM_Ask3_11[a],SUM_Ask3_11[a+4]/SUM_Ask3_11[a]])

In [154]:
a = 5
Ask3_11_2 = pd.DataFrame([SUM_Ask3_11[a],SUM_Ask3_11[a+1]/SUM_Ask3_11[a],SUM_Ask3_11[a+2]/SUM_Ask3_11[a],SUM_Ask3_11[a+3]/SUM_Ask3_11[a],SUM_Ask3_11[a+4]/SUM_Ask3_11[a]])

In [155]:
a = 10
Ask3_11_3 = pd.DataFrame([SUM_Ask3_11[a],SUM_Ask3_11[a+1]/SUM_Ask3_11[a],SUM_Ask3_11[a+2]/SUM_Ask3_11[a],SUM_Ask3_11[a+3]/SUM_Ask3_11[a],SUM_Ask3_11[a+4]/SUM_Ask3_11[a]])

In [156]:
a = 15
Ask3_11_4 = pd.DataFrame([SUM_Ask3_11[a],SUM_Ask3_11[a+1]/SUM_Ask3_11[a],SUM_Ask3_11[a+2]/SUM_Ask3_11[a],SUM_Ask3_11[a+3]/SUM_Ask3_11[a],SUM_Ask3_11[a+4]/SUM_Ask3_11[a]])

In [157]:
a = 20
Ask3_11_5 = pd.DataFrame([SUM_Ask3_11[a],SUM_Ask3_11[a+1]/SUM_Ask3_11[a],SUM_Ask3_11[a+2]/SUM_Ask3_11[a],SUM_Ask3_11[a+3]/SUM_Ask3_11[a],SUM_Ask3_11[a+4]/SUM_Ask3_11[a]])

In [158]:
pd.concat([Ask3_11_1,Ask3_11_2,Ask3_11_3,Ask3_11_4,Ask3_11_5])

,0,1,2,3,4
0,375.000000,1187.000000,4205.000000,872.000000,435.000000
1,0.112000,0.202190,0.079191,0.167431,0.137931
2,-3.357333,-1.887110,-2.169798,-1.712156,-3.685057
3,8.445333,7.422915,6.913912,6.876147,8.142529
4,1749.856000,817.381634,861.297979,827.846330,11.211494
0,546.000000,2056.000000,4919.000000,1119.000000,392.000000
1,0.249084,0.233463,0.106119,0.158177,0.117347
2,-2.538462,-1.013619,-1.242529,-1.492404,-3.507653
3,8.142857,6.636187,6.238463,6.599643,8.528061
4,1360.664835,809.842412,849.940435,817.303843,26.882653


In [160]:
pd.concat([Ask3_11_1,Ask3_11_2,Ask3_11_3,Ask3_11_4,Ask3_11_5]).to_csv('ask3_11_218_227.csv')

In [161]:
a = 0
Ask1_13_1 = pd.DataFrame([SUM_Ask1_13[a],SUM_Ask1_13[a+1]/SUM_Ask1_13[a],SUM_Ask1_13[a+2]/SUM_Ask1_13[a],SUM_Ask1_13[a+3]/SUM_Ask1_13[a],SUM_Ask1_13[a+4]/SUM_Ask1_13[a]])

In [162]:
a = 5
Ask1_13_2 = pd.DataFrame([SUM_Ask1_13[a],SUM_Ask1_13[a+1]/SUM_Ask1_13[a],SUM_Ask1_13[a+2]/SUM_Ask1_13[a],SUM_Ask1_13[a+3]/SUM_Ask1_13[a],SUM_Ask1_13[a+4]/SUM_Ask1_13[a]])

In [163]:
a = 10
Ask1_13_3 = pd.DataFrame([SUM_Ask1_13[a],SUM_Ask1_13[a+1]/SUM_Ask1_13[a],SUM_Ask1_13[a+2]/SUM_Ask1_13[a],SUM_Ask1_13[a+3]/SUM_Ask1_13[a],SUM_Ask1_13[a+4]/SUM_Ask1_13[a]])

In [164]:
a = 15
Ask1_13_4 = pd.DataFrame([SUM_Ask1_13[a],SUM_Ask1_13[a+1]/SUM_Ask1_13[a],SUM_Ask1_13[a+2]/SUM_Ask1_13[a],SUM_Ask1_13[a+3]/SUM_Ask1_13[a],SUM_Ask1_13[a+4]/SUM_Ask1_13[a]])

In [165]:
a = 20
Ask1_13_5 = pd.DataFrame([SUM_Ask1_13[a],SUM_Ask1_13[a+1]/SUM_Ask1_13[a],SUM_Ask1_13[a+2]/SUM_Ask1_13[a],SUM_Ask1_13[a+3]/SUM_Ask1_13[a],SUM_Ask1_13[a+4]/SUM_Ask1_13[a]])

In [166]:
pd.concat([Ask1_13_1,Ask1_13_2,Ask1_13_3,Ask1_13_4,Ask1_13_5])

,0,1,2,3,4
0,8.000000,133.000000,11906.000000,1235.000000,13.000000
1,1.000000,0.924812,0.215942,0.302834,0.153846
2,1.000000,0.984962,0.099194,-0.462348,-1.153846
3,1.875000,2.187970,1.343692,2.148988,2.923077
4,1891.875000,426.857143,811.246262,694.801619,100.307692
0,19.000000,297.000000,2771.000000,1207.000000,6.000000
1,1.000000,0.686869,0.407434,0.346313,0.000000
2,1.000000,0.855219,0.345724,0.106877,0.000000
3,1.578947,2.101010,2.048358,2.241922,2.000000
4,713.842105,472.006734,722.848791,685.996686,0.000000


In [167]:
pd.concat([Ask1_13_1,Ask1_13_2,Ask1_13_3,Ask1_13_4,Ask1_13_5]).to_csv('ask1_13_218_227.csv')

In [168]:
a = 0
Ask2_13_1 = pd.DataFrame([SUM_Ask2_13[a],SUM_Ask2_13[a+1]/SUM_Ask2_13[a],SUM_Ask2_13[a+2]/SUM_Ask2_13[a],SUM_Ask2_13[a+3]/SUM_Ask2_13[a],SUM_Ask2_13[a+4]/SUM_Ask2_13[a]])

In [169]:
a = 5
Ask2_13_2 =pd.DataFrame([SUM_Ask2_13[a],SUM_Ask2_13[a+1]/SUM_Ask2_13[a],SUM_Ask2_13[a+2]/SUM_Ask2_13[a],SUM_Ask2_13[a+3]/SUM_Ask2_13[a],SUM_Ask2_13[a+4]/SUM_Ask2_13[a]])

In [170]:
a = 10
Ask2_13_3 =pd.DataFrame([SUM_Ask2_13[a],SUM_Ask2_13[a+1]/SUM_Ask2_13[a],SUM_Ask2_13[a+2]/SUM_Ask2_13[a],SUM_Ask2_13[a+3]/SUM_Ask2_13[a],SUM_Ask2_13[a+4]/SUM_Ask2_13[a]])

In [171]:
a = 15
Ask2_13_4 =pd.DataFrame([SUM_Ask2_13[a],SUM_Ask2_13[a+1]/SUM_Ask2_13[a],SUM_Ask2_13[a+2]/SUM_Ask2_13[a],SUM_Ask2_13[a+3]/SUM_Ask2_13[a],SUM_Ask2_13[a+4]/SUM_Ask2_13[a]])

In [172]:
a = 20
Ask2_13_5 =pd.DataFrame([SUM_Ask2_13[a],SUM_Ask2_13[a+1]/SUM_Ask2_13[a],SUM_Ask2_13[a+2]/SUM_Ask2_13[a],SUM_Ask2_13[a+3]/SUM_Ask2_13[a],SUM_Ask2_13[a+4]/SUM_Ask2_13[a]])

In [173]:
pd.concat([Ask2_13_1,Ask2_13_2,Ask2_13_3,Ask2_13_4,Ask2_13_5])

,0,1,2,3,4
0,8.000000,133.000000,11906.000000,1235.000000,13.000000
1,1.000000,0.796992,0.047791,0.148988,0.153846
2,2.000000,1.706767,0.050059,-0.436437,-1.000000
3,2.875000,3.263158,2.417521,3.165992,3.923077
4,1908.875000,524.270677,877.442802,823.653441,103.846154
0,19.000000,297.000000,2771.000000,1207.000000,6.000000
1,1.000000,0.609428,0.278239,0.269263,0.000000
2,2.000000,1.478114,0.686756,0.304888,0.000000
3,2.578947,3.134680,3.064237,3.293289,3.000000
4,933.947368,554.488215,800.675208,767.821044,0.000000


In [174]:
pd.concat([Ask2_13_1,Ask2_13_2,Ask2_13_3,Ask2_13_4,Ask2_13_5]).to_csv('ask2_13_218_227.csv')

In [175]:
a = 0
Ask3_13_1 = pd.DataFrame([SUM_Ask3_13[a],SUM_Ask3_13[a+1]/SUM_Ask3_13[a],SUM_Ask3_13[a+2]/SUM_Ask3_13[a],SUM_Ask3_13[a+3]/SUM_Ask3_13[a],SUM_Ask3_13[a+4]/SUM_Ask3_13[a]])

In [176]:
a = 5
Ask3_13_2 = pd.DataFrame([SUM_Ask3_13[a],SUM_Ask3_13[a+1]/SUM_Ask3_13[a],SUM_Ask3_13[a+2]/SUM_Ask3_13[a],SUM_Ask3_13[a+3]/SUM_Ask3_13[a],SUM_Ask3_13[a+4]/SUM_Ask3_13[a]])

In [177]:
a = 10
Ask3_13_3 = pd.DataFrame([SUM_Ask3_13[a],SUM_Ask3_13[a+1]/SUM_Ask3_13[a],SUM_Ask3_13[a+2]/SUM_Ask3_13[a],SUM_Ask3_13[a+3]/SUM_Ask3_13[a],SUM_Ask3_13[a+4]/SUM_Ask3_13[a]])

In [178]:
a = 15
Ask3_13_4 = pd.DataFrame([SUM_Ask3_13[a],SUM_Ask3_13[a+1]/SUM_Ask3_13[a],SUM_Ask3_13[a+2]/SUM_Ask3_13[a],SUM_Ask3_13[a+3]/SUM_Ask3_13[a],SUM_Ask3_13[a+4]/SUM_Ask3_13[a]])

In [179]:
a = 20
Ask3_13_5 = pd.DataFrame([SUM_Ask3_13[a],SUM_Ask3_13[a+1]/SUM_Ask3_13[a],SUM_Ask3_13[a+2]/SUM_Ask3_13[a],SUM_Ask3_13[a+3]/SUM_Ask3_13[a],SUM_Ask3_13[a+4]/SUM_Ask3_13[a]])

In [180]:
pd.concat([Ask3_13_1,Ask3_13_2,Ask3_13_3,Ask3_13_4,Ask3_13_5])

,0,1,2,3,4
0,8.000000,133.000000,11906.000000,1235.000000,13.000000
1,1.000000,0.308271,0.040988,0.083401,0.000000
2,3.000000,1.586466,0.089115,-0.381377,-1.000000
3,3.875000,4.443609,3.418109,4.165992,4.923077
4,2215.875000,843.150376,884.454225,868.249393,0.000000
0,19.000000,297.000000,2771.000000,1207.000000,6.000000
1,1.000000,0.077441,0.239264,0.140845,0.000000
2,3.000000,1.127946,0.915915,0.386081,0.000000
3,3.578947,4.383838,4.068206,4.296603,4.000000
4,988.578947,877.531987,834.661855,841.931234,0.000000


In [181]:
pd.concat([Ask3_13_1,Ask3_13_2,Ask3_13_3,Ask3_13_4,Ask3_13_5]).to_csv('ask3_13_218_227.csv')

In [182]:
a = 0
Ask1_16_1 = pd.DataFrame([SUM_Ask1_16[a],SUM_Ask1_16[a+1]/SUM_Ask1_16[a],SUM_Ask1_16[a+2]/SUM_Ask1_16[a],SUM_Ask1_16[a+3]/SUM_Ask1_16[a],SUM_Ask1_16[a+4]/SUM_Ask1_16[a]])

In [183]:
a = 5
Ask1_16_2 = pd.DataFrame([SUM_Ask1_16[a],SUM_Ask1_16[a+1]/SUM_Ask1_16[a],SUM_Ask1_16[a+2]/SUM_Ask1_16[a],SUM_Ask1_16[a+3]/SUM_Ask1_16[a],SUM_Ask1_16[a+4]/SUM_Ask1_16[a]])

In [184]:
a = 10
Ask1_16_3 = pd.DataFrame([SUM_Ask1_16[a],SUM_Ask1_16[a+1]/SUM_Ask1_16[a],SUM_Ask1_16[a+2]/SUM_Ask1_16[a],SUM_Ask1_16[a+3]/SUM_Ask1_16[a],SUM_Ask1_16[a+4]/SUM_Ask1_16[a]])

In [185]:
a = 15
Ask1_16_4 = pd.DataFrame([SUM_Ask1_16[a],SUM_Ask1_16[a+1]/SUM_Ask1_16[a],SUM_Ask1_16[a+2]/SUM_Ask1_16[a],SUM_Ask1_16[a+3]/SUM_Ask1_16[a],SUM_Ask1_16[a+4]/SUM_Ask1_16[a]])

In [186]:
a = 20
Ask1_16_5 = pd.DataFrame([SUM_Ask1_16[a],SUM_Ask1_16[a+1]/SUM_Ask1_16[a],SUM_Ask1_16[a+2]/SUM_Ask1_16[a],SUM_Ask1_16[a+3]/SUM_Ask1_16[a],SUM_Ask1_16[a+4]/SUM_Ask1_16[a]])

In [187]:
pd.concat([Ask1_16_1,Ask1_16_2,Ask1_16_3,Ask1_16_4,Ask1_16_5])

,0,1,2,3,4
0,47.000000,1458.000000,7747.000000,1522.000000,389.000000
1,1.000000,0.518519,0.220602,0.264126,0.411311
2,1.000000,-0.537037,-0.760475,-1.080815,-0.501285
3,2.489362,3.448560,3.131277,3.498686,2.925450
4,4756.893617,622.137860,735.104040,732.124836,40.776350
0,67.000000,2108.000000,11021.000000,1571.000000,373.000000
1,1.000000,0.476281,0.245622,0.356461,0.672922
2,1.343284,-0.595351,-0.417821,-0.478039,-0.117962
3,3.194030,3.472011,2.746393,2.955442,2.638070
4,2027.671642,654.778937,740.958806,708.577976,81.954424


In [188]:
pd.concat([Ask1_16_1,Ask1_16_2,Ask1_16_3,Ask1_16_4,Ask1_16_5]).to_csv('ask1_16_218_227.csv')

In [189]:
a = 0
Ask2_16_1 = pd.DataFrame([SUM_Ask2_16[a],SUM_Ask2_16[a+1]/SUM_Ask2_16[a],SUM_Ask2_16[a+2]/SUM_Ask2_16[a],SUM_Ask2_16[a+3]/SUM_Ask2_16[a],SUM_Ask2_16[a+4]/SUM_Ask2_16[a]])

In [190]:
a = 5
Ask2_16_2 = pd.DataFrame([SUM_Ask2_16[a],SUM_Ask2_16[a+1]/SUM_Ask2_16[a],SUM_Ask2_16[a+2]/SUM_Ask2_16[a],SUM_Ask2_16[a+3]/SUM_Ask2_16[a],SUM_Ask2_16[a+4]/SUM_Ask2_16[a]])

In [191]:
a = 10
Ask2_16_3 = pd.DataFrame([SUM_Ask2_16[a],SUM_Ask2_16[a+1]/SUM_Ask2_16[a],SUM_Ask2_16[a+2]/SUM_Ask2_16[a],SUM_Ask2_16[a+3]/SUM_Ask2_16[a],SUM_Ask2_16[a+4]/SUM_Ask2_16[a]])

In [192]:
a = 15
Ask2_16_4 = pd.DataFrame([SUM_Ask2_16[a],SUM_Ask2_16[a+1]/SUM_Ask2_16[a],SUM_Ask2_16[a+2]/SUM_Ask2_16[a],SUM_Ask2_16[a+3]/SUM_Ask2_16[a],SUM_Ask2_16[a+4]/SUM_Ask2_16[a]])

In [193]:
a = 20
Ask2_16_5 = pd.DataFrame([SUM_Ask2_16[a],SUM_Ask2_16[a+1]/SUM_Ask2_16[a],SUM_Ask2_16[a+2]/SUM_Ask2_16[a],SUM_Ask2_16[a+3]/SUM_Ask2_16[a],SUM_Ask2_16[a+4]/SUM_Ask2_16[a]])

In [194]:
pd.concat([Ask2_16_1,Ask2_16_2,Ask2_16_3,Ask2_16_4,Ask2_16_5])

,0,1,2,3,4
0,47.000000,1458.000000,7747.000000,1522.000000,389.000000
1,0.978723,0.204390,0.088034,0.107096,0.377892
2,1.978723,-0.617284,-0.726036,-0.971748,-0.182519
3,3.702128,4.575446,4.185104,4.572930,3.925450
4,5042.638298,786.087791,805.186266,809.626150,59.850900
0,67.000000,2108.000000,11021.000000,1571.000000,373.000000
1,0.820896,0.173624,0.086018,0.181413,0.619303
2,1.955224,-0.852941,-0.342056,-0.341820,0.404826
3,4.388060,4.646110,3.815806,3.986633,3.707775
4,2419.477612,819.643264,811.013701,785.201146,98.069705


In [195]:
pd.concat([Ask2_16_1,Ask2_16_2,Ask2_16_3,Ask2_16_4,Ask2_16_5]).to_csv('ask2_16_218_227.csv')

In [196]:
a = 0
Ask3_16_1 = pd.DataFrame([SUM_Ask3_16[a],SUM_Ask3_16[a+1]/SUM_Ask3_16[a],SUM_Ask3_16[a+2]/SUM_Ask3_16[a],SUM_Ask3_16[a+3]/SUM_Ask3_16[a],SUM_Ask3_16[a+4]/SUM_Ask3_16[a]])

In [197]:
a = 5
Ask3_16_2 = pd.DataFrame([SUM_Ask3_16[a],SUM_Ask3_16[a+1]/SUM_Ask3_16[a],SUM_Ask3_16[a+2]/SUM_Ask3_16[a],SUM_Ask3_16[a+3]/SUM_Ask3_16[a],SUM_Ask3_16[a+4]/SUM_Ask3_16[a]])

In [198]:
a = 10
Ask3_16_3 = pd.DataFrame([SUM_Ask3_16[a],SUM_Ask3_16[a+1]/SUM_Ask3_16[a],SUM_Ask3_16[a+2]/SUM_Ask3_16[a],SUM_Ask3_16[a+3]/SUM_Ask3_16[a],SUM_Ask3_16[a+4]/SUM_Ask3_16[a]])

In [199]:
a = 15
Ask3_16_4 = pd.DataFrame([SUM_Ask3_16[a],SUM_Ask3_16[a+1]/SUM_Ask3_16[a],SUM_Ask3_16[a+2]/SUM_Ask3_16[a],SUM_Ask3_16[a+3]/SUM_Ask3_16[a],SUM_Ask3_16[a+4]/SUM_Ask3_16[a]])

In [200]:
a = 20
Ask3_16_5 = pd.DataFrame([SUM_Ask3_16[a],SUM_Ask3_16[a+1]/SUM_Ask3_16[a],SUM_Ask3_16[a+2]/SUM_Ask3_16[a],SUM_Ask3_16[a+3]/SUM_Ask3_16[a],SUM_Ask3_16[a+4]/SUM_Ask3_16[a]])

In [201]:
pd.concat([Ask3_16_1,Ask3_16_2,Ask3_16_3,Ask3_16_4,Ask3_16_5])

,0,1,2,3,4
0,47.000000,1458.000000,7747.000000,1522.000000,389.000000
1,0.957447,0.061043,0.057571,0.047306,0.352185
2,2.893617,-0.909465,-0.711966,-0.944152,0.141388
3,4.702128,5.681756,5.195818,5.572930,4.935733
4,5352.553191,871.867627,819.854524,845.846912,78.953728
0,67.000000,2108.000000,11021.000000,1571.000000,373.000000
1,0.820896,0.031784,0.059886,0.110121,0.576408
2,2.776119,-1.128083,-0.301679,-0.254615,0.849866
3,5.388060,5.774668,4.817076,4.986633,4.823056
4,2704.074627,884.443548,827.498503,823.371101,119.458445


In [203]:
pd.concat([Ask3_16_1,Ask3_16_2,Ask3_16_3,Ask3_16_4,Ask3_16_5]).to_csv('ask3_16_218_227.csv')